In [144]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [103]:
# loading data frame into pandas df
credit_card_data= pd.read_csv('/creditcard.csv')
credit_card_data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [104]:
credit_card_data.tail(5)
ccd= credit_card_data
ccd.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [105]:
#checking number of null values
ccd.isnull().sum() #no missing value present


,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [106]:
#checking the distribution of fraud and legit transaction
ccd['Class'].value_counts() #very unbalanced data 0-> legit transaction, 1-> fraud transaction

,count
Class,
0,284315
1,492


In [107]:
legit= ccd[ccd.Class ==0]
fraud = ccd[ccd.Class==1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [108]:
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [109]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [110]:
#mean for fraud transactions was pretty higher than for legit transaction

ccd.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Number of fraud transactions = 492
Number of legit transactions= 284315

Creating a uniform dataset by taking a random sample of 492 entries from legit and combining it with 492 fraud entries

In [111]:
legit_sample= legit.sample(n=492)

In [112]:
#combining legit_sample and fraud ds

new= pd.concat([legit_sample, fraud], axis=0) #axis=0 row wise, axis=1 column wise

In [113]:
new.head(5)
new.tail(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
274382,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,-1.076274,...,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1
274475,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,-1.745854,...,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1
275992,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,-0.063434,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
276071,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,-0.140927,...,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1
276864,167338.0,-1.374424,2.793185,-4.346572,2.400731,-1.688433,0.111136,-0.922038,-2.149930,-2.027474,...,-0.870779,0.504849,0.137994,0.368275,0.103137,-0.414209,0.454982,0.096711,349.08,1
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [114]:
 new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984 entries, 196488 to 281674
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    984 non-null    float64
 1   V1      984 non-null    float64
 2   V2      984 non-null    float64
 3   V3      984 non-null    float64
 4   V4      984 non-null    float64
 5   V5      984 non-null    float64
 6   V6      984 non-null    float64
 7   V7      984 non-null    float64
 8   V8      984 non-null    float64
 9   V9      984 non-null    float64
 10  V10     984 non-null    float64
 11  V11     984 non-null    float64
 12  V12     984 non-null    float64
 13  V13     984 non-null    float64
 14  V14     984 non-null    float64
 15  V15     984 non-null    float64
 16  V16     984 non-null    float64
 17  V17     984 non-null    float64
 18  V18     984 non-null    float64
 19  V19     984 non-null    float64
 20  V20     984 non-null    float64
 21  V21     984 non-null    float64
 22 

In [115]:
new['Class'].value_counts()

,count
Class,
0,492
1,492


In [116]:
new.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95457.802846,-0.107872,0.045838,0.102351,0.058267,0.003006,0.136917,-0.038036,0.027984,0.001185,...,-0.037791,-0.007399,0.035319,-0.019112,0.002486,0.020775,-0.018751,0.007780,-0.004385,87.055325
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the data into features & targets
Target-> Class

In [117]:
X = new.drop(columns='Class', axis=1)
Y = new['Class' ]
X.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
196488,131535.0,1.984394,-1.875270,-0.724092,-1.317574,-1.754866,-0.870269,-1.030945,-0.268715,-1.067613,...,-0.123479,-0.050095,0.053669,0.126464,0.037330,-0.336711,-0.195307,-0.001139,-0.017672,167.00
89637,62662.0,0.822913,-0.681764,0.914696,0.517370,-0.139458,2.277249,-0.967891,0.822099,0.670367,...,-0.056027,0.105153,0.433097,0.036136,-1.298210,-0.075524,0.439889,0.068994,0.012709,87.84
111914,72427.0,-1.079493,0.361900,1.308379,-1.653535,-0.969706,-0.884095,-0.010407,0.454676,-1.748892,...,-0.325192,-0.308286,-0.675583,0.116489,0.520102,-0.252989,0.747099,0.117162,0.070821,50.27
152117,96913.0,2.123803,0.080781,-2.315962,-0.065928,0.980549,-0.758940,0.497985,-0.373201,1.401393,...,-0.276063,0.002029,0.264361,-0.068143,0.164512,0.439156,0.183655,-0.130226,-0.094429,11.93
155879,106684.0,1.500942,-1.953883,-0.867731,-0.680231,-0.931363,0.425190,-0.849255,-0.063441,0.989310,...,0.734565,0.485015,0.911373,-0.188871,0.239974,-0.282860,-0.164221,-0.074423,0.001770,350.25


In [118]:
print(Y)

196488    0
89637     0
111914    0
152117    0
155879    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Train-test split
Test size->0.2== 20% data will be test data, 80% train data
stratify= Y--> evenly

In [119]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [120]:
print(X_train.shape)
print(X_test.shape)
print(X.shape)

(787, 30)
(197, 30)
(984, 30)


Training the model with Logistics Regression (as this is a binary classification model)

In [121]:
model= LogisticRegression()

Training the Logistics Regression model with training data

In [122]:
model.fit(X_train,Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation
1. Training data

In [123]:
X_train_prediction= model.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [124]:
print(training_data_accuracy)

0.9491740787801779


On training data, 94% accuracy achieved

In [125]:
X_test_prediction= model.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [126]:
print(test_data_accuracy)

0.934010152284264


In [142]:
con_matrix = confusion_matrix(Y_test, X_test_prediction)
print("Confusion Matrix:")
print(con_matrix)
class_report = classification_report(Y_test, X_test_prediction)
print("\nClassification Report:")
print(class_report)



Confusion Matrix:
[[95  4]
 [ 9 89]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.96      0.94        99
           1       0.96      0.91      0.93        98

    accuracy                           0.93       197
   macro avg       0.94      0.93      0.93       197
weighted avg       0.94      0.93      0.93       197



On test data, accuracy is 93%

If accuracy on training data is different than on test data, then it means model is overfitted or underfitted
If training accuracy is way higher than test accuracy --> overfitted
If test data accuracy is high -->underfitted

Using Decision Tree Model

In [129]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, Y_train)
dt_train_pred = dt_model.predict(X_train)
dt_train_accuracy = accuracy_score(Y_train, dt_train_pred)
print(dt_train_accuracy)

1.0


In [131]:
dt_test_pred = dt_model.predict(X_test)
dt_test_accuracy = accuracy_score(Y_test, dt_test_pred)
print(dt_test_accuracy)

0.9086294416243654


In [143]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, Y_train)

y_pred_dt = dt_model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(Y_test, y_pred_dt))

print("\nClassification Report:")
print(classification_report(Y_test, y_pred_dt, digits=2))

Confusion Matrix:
[[92  7]
 [11 87]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        99
           1       0.93      0.89      0.91        98

    accuracy                           0.91       197
   macro avg       0.91      0.91      0.91       197
weighted avg       0.91      0.91      0.91       197



Using Random Forest Model

In [137]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, Y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)


train_acc = accuracy_score(Y_train, y_train_pred)
test_acc = accuracy_score(Y_test, y_test_pred)

print("Confusion Matrix:\n", confusion_matrix(Y_test, y_test_pred))
print("\nClassification Report:", classification_report(Y_test, y_test_pred))

Confusion Matrix:
 [[98  1]
 [13 85]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.99      0.93        99
           1       0.99      0.87      0.92        98

    accuracy                           0.93       197
   macro avg       0.94      0.93      0.93       197
weighted avg       0.94      0.93      0.93       197



Using KNN Model

In [141]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, Y_train)

knn_train_pred = knn_model.predict(X_train)
knn_test_pred = knn_model.predict(X_test)

knn_train_accuracy = accuracy_score(Y_train, knn_train_pred)
knn_test_accuracy = accuracy_score(Y_test, knn_test_pred)

print(knn_train_accuracy)
print(knn_test_accuracy)

conf_matrix = confusion_matrix(Y_test, knn_test_pred)
class_report = classification_report(Y_test, knn_test_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


0.7623888182973316
0.5939086294416244
Confusion Matrix:
 [[54 45]
 [35 63]]

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.55      0.57        99
           1       0.58      0.64      0.61        98

    accuracy                           0.59       197
   macro avg       0.60      0.59      0.59       197
weighted avg       0.60      0.59      0.59       197

